# Import libraries

In [1]:
import os
os.getcwd()

'c:\\Users\\anjik\\Desktop\\MLOPs_projects\\Gen_AI\\Medical-Chatbot_Llama2\\notebook'

In [2]:
os.chdir("../")

In [3]:
os.getcwd()

'c:\\Users\\anjik\\Desktop\\MLOPs_projects\\Gen_AI\\Medical-Chatbot_Llama2'

In [4]:
from langchain.chains import RetrievalQA # chain for Q&A against an vector store index
from langchain.embeddings import HuggingFaceEmbeddings # wrapper around sentense_transformers embedding models
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

# to load pdf files
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

# chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

from dotenv import load_dotenv

c:\Users\anjik\anaconda3\envs\medibot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## PINECONE API KEY from .env

In [5]:

import os
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_API_KEY

'pcsk_5J7nq2_sA7PCqEktiF4KxRt32Yo7FhxeB3Fx4QqpztFD6WKBos4ruF4wAmi5vPZjEVRBK'

# Load pdf file, extract data

In [6]:
def load_data(data):
    loader= DirectoryLoader(path=data, 
                               glob="*.pdf", 
                               loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [7]:
documents= load_data("data/")
documents[0:5]


[Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 1}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 2}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 3}, page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow,Manager, Imaging and Multimedia\nContent\nRobyn V . Young,Project Manager, Imaging and\nMultimedia Content\nDean Daup

In [8]:
print("page_number:", documents[0].metadata["page"])

page_number: 0


# Chunking data

In [9]:
# create function to make text chunks

def text_split(documents):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(documents)
    return text_chunks

In [10]:
text_chunks= text_split(documents)
print(len(text_chunks))
text_chunks[0:10]

5860


[Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 1}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 2}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 3}, page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow,Manager, Imaging and Multimedia\nContent\nRobyn V . Young,Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and'),
 Document(metadata={'source': 'data\\Medical_boo

# Store chunks in vector DB ---> before that convert chunks into vector representation i.e Embedding the data

- using embedding model: "sentence-transformers/all-MiniLM-L6-v2"
- It is a sentence transformers model, maps sentenses and paragraphs to a `384` dimensional dense vector --> used for clustering and/or semantic search

In [11]:
# download embedding model

def download_huggingface_embedding_model():
    embedding_model= HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embedding_model

In [12]:
embedding_model= download_huggingface_embedding_model()
embedding_model

C:\Users\anjik\AppData\Local\Temp\ipykernel_3800\1011557484.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model= HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [14]:
# embedding test
query_result= embedding_model.embed_query("Hello world")
print("Length of embeddings:", len(query_result))

Length of embeddings: 384


# Initialize pinecone, create index, saves chunks and embeddings into index of pinecone DB 



In [15]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os


pinecone_obj = Pinecone(api_key=PINECONE_API_KEY)


index_name = "medical-chatbot-vector-index"

pinecone_obj.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [16]:
Pinecone.init(api_key= PINECONE_API_KEY,
              environment= PINECONE_API_ENV)
# your pinecone index name
index_name= "medical_chatbot"

# create embeddings for each of teh text chunks, store
docsearch= Pinecone.from_texts([t.page_content for t in text_chunks], embedding_model, index_name=index_name)

AttributeError: type object 'PineconeGRPC' has no attribute 'init'

# Give query, get rank results from Pinecone knowledge base

In [ ]:
# load index from pinecone
docsearch= Pinecone.from_existing_index(index_name, embedding)
query= "What are allergies?"

# get top 3 similarity searches
docs= docsearch.similarity_search(query, k=3)
print("Result", docs)

# Give rank results to LLM

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
prompt= PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs= {"prompt":prompt}

In [14]:
# load llama model
llm= CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                   model_type="llama",
                   config={"max_new_tokens":512,
                           "temperature":0.8})

# create Q&A Retrival: 

In [ ]:
qa= RetrievalQA.from_chain_type(llm=llm, 
                                chain_type= "stuff",
                                retriever= docsearch.as_retriever(search_kwargs= {'k':2}),
                                return_source_documents=True,
                                chain_type_kwargs=chain_type_kwargs)


In [ ]:
while True:
    user_input= input(f"Input Prompt:")
    result= qa({"query": user_input})
    print("Response:", result["result"])